In [16]:
import json
from itertools import groupby
from collections import defaultdict
import os
import pandas as pd

In [18]:
files = list(os.walk("data"))[0][2]

dfs = []

for file in files:
    file_path = f"data/{file}"
    print(file_path)
    with open(file_path,"r") as f:
        raw_data = json.load(f)
        data = []

        for item in raw_data:
            if type(item) == list:
                data.extend(item)
            else:
                data.append(item)
                
    result = defaultdict(lambda: defaultdict(int))

    for i in data:
        keyword = i["keyword"]
        comments = i["num_comments"]
        if i["type"] == "posts":
            result[keyword]["num_keyword_posts"] += 1
            result[keyword]["num_comments"] += comments
        elif i["type"] == "comments":
            result[keyword]["num_keyword_comments"] += 1
    df = pd.DataFrame(result).T
    df["date"] = file_path.split("_")[1].split(".")[0]
        
    dfs.append(df)



data/data_21-02-14.json


In [19]:
stat_df = pd.concat(dfs).reset_index().rename(columns={"index":"keyword"}).sort_values(["keyword","date"])
stat_df = stat_df[["date","keyword","num_keyword_posts","num_comments","num_keyword_comments"]]
stat_df

,date,keyword,num_keyword_posts,num_comments,num_keyword_comments
0,21-02-14,BTC,1343,6094,628
1,21-02-14,Bitcoin,2568,11768,709


In [4]:
stat_df.to_csv("reddit_crypto_stat.csv",index=False)